In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
commute = pd.read_csv('../data/census_data/2021_Data.csv')
#commute

Let's get rid of those pesky Error and Percent Allocated columns!

In [3]:
pattern  = re.compile('Margin of Error')
pattern2 = re.compile('PERCENT ALLOCATED')
drop = []

for col in commute.columns:
    if re.search(pattern,col):
        #print(col)
        drop.append(col)
    elif re.search(pattern2,col):
        #print(col)
        drop.append(col)
#end

In [4]:
commute = commute.drop(columns = drop)

And the random Unnamed column while we're at it.

In [5]:
commute = commute.drop(columns = 'Unnamed: 778')
#commute

Having the letter "N" doesn't help... let's replace it with NaN.

In [6]:
commute = commute.replace(to_replace='N',value=np.nan)
#commute

And let's clean up those columns names! There's a lot going on with them.

In [7]:
new_cols = []

for col in commute.columns:
    new_col = col.replace('Estimate!!Total!!', '').replace('!!',' ').strip()
    new_cols.append(new_col)

In [8]:
new_cols

['Geography',
 'Geographic Area Name',
 'Workers 16 years and over',
 'Workers 16 years and over AGE 16 to 19 years',
 'Workers 16 years and over AGE 20 to 24 years',
 'Workers 16 years and over AGE 25 to 44 years',
 'Workers 16 years and over AGE 45 to 54 years',
 'Workers 16 years and over AGE 55 to 59 years',
 'Workers 16 years and over AGE 60 years and over',
 'Workers 16 years and over AGE Median age (years)',
 'Workers 16 years and over SEX Male',
 'Workers 16 years and over SEX Female',
 'Workers 16 years and over RACE AND HISPANIC OR LATINO ORIGIN One race',
 'Workers 16 years and over RACE AND HISPANIC OR LATINO ORIGIN One race White',
 'Workers 16 years and over RACE AND HISPANIC OR LATINO ORIGIN One race Black or African American',
 'Workers 16 years and over RACE AND HISPANIC OR LATINO ORIGIN One race American Indian and Alaska Native',
 'Workers 16 years and over RACE AND HISPANIC OR LATINO ORIGIN One race Asian',
 'Workers 16 years and over RACE AND HISPANIC OR LATINO ORI

In [9]:
commute.columns = new_cols
#commute

Pull the columns strictly related to commute times.

In [10]:
pattern = re.compile('TRAVEL TIME')
drop = []

for col in commute.columns:
    if (not re.search(pattern,col)) and ((col != 'Geographic Area Name') and (col != 'Workers 16 years and over')
                                        and (col != 'Workers 16 years and over who did not work from home') # These columns needed to stay
                                        and (col != 'Estimate Car, truck, or van -- drove alone Workers 16 years and over')
                                        and (col != 'Estimate Car, truck, or van -- carpooled Workers 16 years and over')
                                        and (col != 'Estimate Public transportation (excluding taxicab) Workers 16 years and over')):
        drop.append(col)

commutes = commute.drop(columns = drop)

In [11]:
#commutes

In [12]:
commutes = commutes.set_index('Geographic Area Name')
#commutes

Checking my home county just to make sure it works as intended.

In [13]:
commutes.loc['Montgomery County, Maryland']

Workers 16 years and over                                                                                                                                         534726
Workers 16 years and over who did not work from home                                                                                                              334392
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes                                                                        7.0
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes                                                                            9.3
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes                                                                           13.6
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes                                                                  

In [14]:
commutes.dtypes

Workers 16 years and over                                                                                                                                         object
Workers 16 years and over who did not work from home                                                                                                              object
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes                                                                     object
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes                                                                         object
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes                                                                         object
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes                                                                  

## The data types are wrong. This will have to be something that will be cleaned up.

And we can't groupby anything because apparently there are '-'  floating around...

In [15]:
commutes = commutes.replace(to_replace = '-', value = np.nan)

In [16]:
commutes = commutes.apply(pd.to_numeric)
commutes.dtypes

Workers 16 years and over                                                                                                                                         float64
Workers 16 years and over who did not work from home                                                                                                              float64
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes                                                                     float64
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes                                                                         float64
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes                                                                         float64
Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes                                                             

In [17]:
commutes.head(12)

,Workers 16 years and over,Workers 16 years and over who did not work from home,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,...,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 60 or more minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes)
Geographic Area Name,,,,,,,,,,,,,,,,,,,,,
"Baldwin County, Alabama",94983.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Calhoun County, Alabama",48870.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Cullman County, Alabama",34266.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"DeKalb County, Alabama",26852.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Elmore County, Alabama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Etowah County, Alabama",36193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Houston County, Alabama",50687.0,47993.0,13.8,14.4,18.8,18.6,8.6,11.7,4.2,5.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Jefferson County, Alabama",364888.0,324856.0,7.8,11.6,15.8,17.1,7.4,16.2,8.1,8.7,...,7.2,32.2,13.5,34.8,5.6,6.7,0.0,0.0,0.0,16.3
"Lauderdale County, Alabama",36540.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


With that done, let's make a 'State' column (we'll group by it later).

In [18]:
states = []

for ind, values in commutes.iterrows():
    state = ind.split(', ')[1]
    #print(state)
    states.append(state)

In [19]:
commutes['State'] = states
#commutes

In [20]:
commutes.groupby('State').mean()

,Workers 16 years and over,Workers 16 years and over who did not work from home,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,...,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 60 or more minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes)
State,,,,,,,,,,,,,,,,,,,,,
Alabama,93176.444444,172789.800000,10.442857,12.157143,17.414286,17.542857,7.442857,14.414286,6.342857,7.871429,...,1.440000,6.440000,33.020000,23.100000,1.120000,7.740000,25.460000,1.680000,0.000000,21.500000
Alaska,79346.666667,135757.000000,14.300000,18.600000,21.100000,16.850000,4.900000,10.250000,3.550000,3.850000,...,37.550000,5.950000,0.000000,1.350000,4.700000,17.800000,1.650000,10.250000,20.700000,26.950000
Arizona,320009.600000,460685.000000,16.420000,16.100000,16.100000,15.120000,6.080000,12.340000,6.100000,5.480000,...,1.100000,27.880000,3.460000,11.260000,1.400000,18.940000,13.240000,4.320000,18.360000,33.520000
Arkansas,83056.300000,228830.000000,13.450000,16.875000,16.850000,16.425000,8.075000,11.600000,6.625000,5.750000,...,0.000000,48.400000,25.500000,8.966667,5.133333,3.933333,0.000000,8.100000,0.000000,20.450000
California,430679.926829,449054.266667,12.682759,14.479310,16.082759,14.893103,6.641379,13.151724,6.162069,7.065517,...,5.541379,3.944828,9.989655,5.293103,5.241379,13.727586,9.155172,18.000000,29.093103,44.534483
Colorado,218717.416667,176498.818182,11.854545,13.890909,16.281818,15.409091,7.345455,14.618182,6.945455,7.409091,...,3.618182,4.654545,4.445455,18.527273,2.700000,14.009091,6.081818,11.909091,34.063636,41.160000
Connecticut,216214.000000,247715.200000,13.416667,14.116667,18.500000,15.250000,7.566667,12.700000,6.683333,6.216667,...,0.466667,5.516667,14.750000,8.416667,5.000000,13.200000,7.700000,8.633333,36.300000,50.766667
Delaware,151559.333333,223006.000000,12.450000,13.750000,16.400000,14.850000,7.500000,11.250000,8.350000,8.950000,...,6.100000,15.850000,0.250000,25.550000,5.250000,22.500000,0.000000,21.000000,3.500000,27.400000
District of Columbia,606015.00000

In [21]:
#pd.set_option('display.max_columns', None)
#commutes

There are a lot of counties that have mostly null values... Let's drop those rows.

In [22]:
commutes_nan = commutes.dropna(thresh = 30)
commutes_nan

,Workers 16 years and over,Workers 16 years and over who did not work from home,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,...,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 60 or more minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes),State
Geographic Area Name,,,,,,,,,,,,,,,,,,,,,
"Houston County, Alabama",50687.0,47993.0,13.8,14.4,18.8,18.6,8.6,11.7,4.2,5.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama
"Jefferson County, Alabama",364888.0,324856.0,7.8,11.6,15.8,17.1,7.4,16.2,8.1,8.7,...,32.2,13.5,34.8,5.6,6.7,0.0,0.0,0.0,16.3,Alabama
"Madison County, Alabama",233667.0,196416.0,9.0,10.8,17.1,18.1,7.5,14.7,8.4,7.3,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,NaN,Alabama
"Mobile County, Alabama",182575.0,168924.0,8.8,9.8,16.4,19.1,6.8,17.1,7.5,8.3,...,0.0,0.0,40.7,0.0,32.0,27.3,0.0,0.0,28.7,Alabama
"Montgomery County, Alabama",133290.0,125760.0,8.4,13.2,16.2,19.9,8.7,16.1,5.0,7.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Waukesha County, Wisconsin",245884.0,204323.0,12.7,13.3,14.7,17.7,9.2,15.5,8.2,5.6,...,9.4,5.5,0.0,0.0,58.3,0.0,0.0,26.8,35.3,Wisconsin
"Winnebago County, Wisconsin",97737.0,86447.0,17.5,17.8,20.0,16.1,5.1,9.8,4.9,3.8,...,0.0,14.5,0.0,0.0,36.5,0.0,5.4,5.3,22.5,Wisconsin
"Wood County, Wisconsin",36785.0,NaN,23.8,16.1,17.2,12.3,5.4,8.8,4.3,5.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NaN,Wisconsin


In [23]:
commutes_nan.groupby('State').mean()

,Workers 16 years and over,Workers 16 years and over who did not work from home,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,...,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 60 or more minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes)
State,,,,,,,,,,,,,,,,,,,,,
Alabama,167936.142857,172789.800000,10.442857,12.157143,17.414286,17.542857,7.442857,14.414286,6.342857,7.871429,...,1.440000,6.440000,33.020000,23.100000,1.120000,7.740000,25.460000,1.680000,0.000000,21.500000
Alaska,94809.500000,135757.000000,14.300000,18.600000,21.100000,16.850000,4.900000,10.250000,3.550000,3.850000,...,37.550000,5.950000,0.000000,1.350000,4.700000,17.800000,1.650000,10.250000,20.700000,26.950000
Arizona,591605.400000,460685.000000,16.420000,16.100000,16.100000,15.120000,6.080000,12.340000,6.100000,5.480000,...,1.100000,27.880000,3.460000,11.260000,1.400000,18.940000,13.240000,4.320000,18.360000,33.520000
Arkansas,150342.500000,228830.000000,13.450000,16.875000,16.850000,16.425000,8.075000,11.600000,6.625000,5.750000,...,0.000000,48.400000,25.500000,8.966667,5.133333,3.933333,0.000000,8.100000,0.000000,20.450000
California,590116.965517,462341.448276,12.682759,14.479310,16.082759,14.893103,6.641379,13.151724,6.162069,7.065517,...,5.541379,3.944828,9.989655,5.293103,5.241379,13.727586,9.155172,18.000000,29.093103,44.534483
Colorado,232219.181818,188229.100000,11.854545,13.890909,16.281818,15.409091,7.345455,14.618182,6.945455,7.409091,...,3.618182,4.654545,4.445455,18.527273,2.700000,14.009091,6.081818,11.909091,34.063636,41.160000
Connecticut,267941.333333,294360.000000,13.416667,14.116667,18.500000,15.250000,7.566667,12.700000,6.683333,6.216667,...,0.466667,5.516667,14.750000,8.416667,5.000000,13.200000,7.700000,8.633333,36.300000,50.766667
Delaware,191087.000000,223006.000000,12.450000,13.750000,16.400000,14.850000,7.500000,11.250000,8.350000,8.950000,...,6.100000,15.850000,0.250000,25.550000,5.250000,22.500000,0.000000,21.000000,3.500000,27.400000
District of Columbia,606015.000

I need to be careful with how I approach this data now. Taking averages of percents can be useful (I literally did a mathematical proof that shows that taking averages of percentages will still yield a row that sums to 100%), but if each sample size is different, percentages will become skewed and meaningless.

Averages are fine for the mean travel times, but other than that, they should be avoided. I would like to get rid of the percentages by multiplying the total number of people to the percentage, but this dataset doesn't actually give me a breakdown of how many people commute, much less which category they fall under. I can maybe try looking for this info, but I'm not sure I would find it. Perhaps I should start focusing somewhere else.

# OF COURSE THE DATA WAS ALREADY IN THE DATASET. I had simply dropped it earlier. *Now revised: See Cell 10*

Well, this should be fun then.

First, I need to find a way to get rid of the percentages and put in the actual number of people. This way, I can take averages without skewing anything.

In [24]:
for row in commutes_nan.itertuples():
    #print('THE ROW INDEX IS:',row.Index)
    
    comm_total  = row[2]
    drove_alone = row[13]
    carpool     = row[24]
    pub_transit = row[35]

    for i in range(3,12):
        #print('\n',i)
        try:
            new_val = int(comm_total*(row[i]/100))
            #print(f'{comm_total} times {row[i]/100} equals {new_val}\t{commutes_nan.columns[i-1]}')
            commutes_nan.loc[row.Index, commutes_nan.columns[i-1]] = new_val
        except:
            #print(f'{comm_total} times {row[i]/100} equals {np.nan}\t{commutes.columns[i-1]}')
            commutes_nan.loc[row.Index, commutes_nan.columns[i-1]] = np.nan
    
    
    for i in range(14,23):
        #print('\n',i)
        try:
            new_val = int(drove_alone*(row[i]/100))
            #print(f'{drove_alone} times {row[i]/100} equals {new_val}\t{commutes_nan.columns[i-1]}')
            commutes_nan.loc[row.Index, commutes_nan.columns[i-1]] = new_val
        except:
            #print(f'{drove_alone} times {row[i]/100} equals {np.nan}\t{commutes_nnan.columns[i-1]}')
            commutes_nan.loc[row.Index, commutes_nan.columns[i-1]] = np.nan
    
    
    for i in range(25,34):
        #print('\n',i)
        try:
            new_val = int(carpool*(row[i]/100))
            #print(f'{carpool} times {row[i]/100} equals {new_val}\t{commutes_nan.columns[i-1]}')
            commutes_nan.loc[row.Index, commutes_nan.columns[i-1]] = new_val
        except:
            #print(f'{carpool} times {row[i]/100} equals {np.nan}\t{commutes_nan.columns[i-1]}')
            commutes_nan.loc[row.Index, commutes_nan.columns[i-1]] = np.nan
            
            
    for i in range(36,45):
        #print('\n',i)
        try:
            new_val = int(pub_transit*(row[i]/100))
            #print(f'{pub_transit} times {row[i]/100} equals {new_val}\t{commutes_nan.columns[i-1]}')
            commutes_nan.loc[row.Index, commutes_nan.columns[i-1]] = new_val
        except:
            #print(f'{pub_transit} times {row[i]/100} equals {np.nan}\t{commutes_nan.columns[i-1]}')
            commutes_nan.loc[row.Index, commutes_nan.columns[i-1]] = np.nan
    

In [25]:
#commutes_nan

Since a lot of these rows are null, I'm gonna drop any that are mostly null.

In [26]:
commutes_nan = commutes.dropna(thresh = 30)
#commutes_nan

Now, let's group by State.

In [27]:
commutes_avg = commutes_nan.groupby('State').mean().astype(int)
commutes_avg

,Workers 16 years and over,Workers 16 years and over who did not work from home,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,...,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 60 or more minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes)
State,,,,,,,,,,,,,,,,,,,,,
Alabama,167936,172789,10,12,17,17,7,14,6,7,...,1,6,33,23,1,7,25,1,0,21
Alaska,94809,135757,14,18,21,16,4,10,3,3,...,37,5,0,1,4,17,1,10,20,26
Arizona,591605,460685,16,16,16,15,6,12,6,5,...,1,27,3,11,1,18,13,4,18,33
Arkansas,150342,228830,13,16,16,16,8,11,6,5,...,0,48,25,8,5,3,0,8,0,20
California,590116,462341,12,14,16,14,6,13,6,7,...,5,3,9,5,5,13,9,18,29,44
Colorado,232219,188229,11,13,16,15,7,14,6,7,...,3,4,4,18,2,14,6,11,34,41
Connecticut,267941,294360,13,14,18,15,7,12,6,6,...,0,5,14,8,5,13,7,8,36,50
Delaware,191087,223006,12,13,16,14,7,11,8,8,...,6,15,0,25,5,22,0,21,3,27
District of Columbia,606015,435044,4,5,8,11,6,17,13,16,...,1,1,3,7,2,17,14,19,32,48


In [28]:
commutes_avg.sort_values('Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes)', ascending = False)

,Workers 16 years and over,Workers 16 years and over who did not work from home,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,...,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 60 or more minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes)
State,,,,,,,,,,,,,,,,,,,,,
District of Columbia,606015,435044,4,5,8,11,6,17,13,16,...,1,1,3,7,2,17,14,19,32,48
Puerto Rico,147539,219197,5,7,15,16,4,19,9,9,...,0,2,16,3,0,22,0,26,27,40
Georgia,276289,272657,8,11,14,14,7,16,8,9,...,13,6,11,10,3,15,1,11,26,41
New York,336097,323162,13,13,15,14,6,12,7,7,...,1,6,8,9,3,17,5,13,33,45
Maryland,270576,235720,9,11,14,14,7,14,9,9,...,6,4,3,9,1,18,3,18,34,48
New Jersey,239076,201391,11,13,15,14,6,13,7,8,...,1,3,5,6,3,23,6,12,36,50
Tennessee,221954,271679,11,12,15,14,7,14,7,8,...,26,1,18,4,2,12,1,18,14,39
New Mexico,168553,284634,14,14,20,13,6,12,4,6,...,36,0,0,8,1,23,3,7,18,41
California,590116,462341,12,14,16,14,6,13,6,7,...,5,3,9,5,5,13,9,18,29,44


Cleaning up the columns names... again.

In [29]:
pattern1 = r'Workers 16 years and over who did not work from home'
pattern2 = r'Estimate'
pattern3 = r'(excluding taxicab)'

new_cols = {}

for col in commutes_avg.columns:
    if (col != 'Workers 16 years and over who did not work from home') and (re.search(pattern1,col) 
                                                                            or re.search(pattern2,col) 
                                                                            or re.search(pattern3,col)):
        new_col = col.replace(str(pattern1),'').replace(str(pattern2),'').replace(str(pattern3),'').strip()
        new_cols[col] = new_col
#end

In [30]:
commutes_avg = commutes_avg.rename(columns = new_cols)
commutes_avg

,Workers 16 years and over,Workers 16 years and over who did not work from home,TRAVEL TIME TO WORK Less than 10 minutes,TRAVEL TIME TO WORK 10 to 14 minutes,TRAVEL TIME TO WORK 15 to 19 minutes,TRAVEL TIME TO WORK 20 to 24 minutes,TRAVEL TIME TO WORK 25 to 29 minutes,TRAVEL TIME TO WORK 30 to 34 minutes,TRAVEL TIME TO WORK 35 to 44 minutes,TRAVEL TIME TO WORK 45 to 59 minutes,...,Public transportation TRAVEL TIME TO WORK Less than 10 minutes,Public transportation TRAVEL TIME TO WORK 10 to 14 minutes,Public transportation TRAVEL TIME TO WORK 15 to 19 minutes,Public transportation TRAVEL TIME TO WORK 20 to 24 minutes,Public transportation TRAVEL TIME TO WORK 25 to 29 minutes,Public transportation TRAVEL TIME TO WORK 30 to 34 minutes,Public transportation TRAVEL TIME TO WORK 35 to 44 minutes,Public transportation TRAVEL TIME TO WORK 45 to 59 minutes,Public transportation TRAVEL TIME TO WORK 60 or more minutes,Public transportation TRAVEL TIME TO WORK Mean travel time to work (minutes)
State,,,,,,,,,,,,,,,,,,,,,
Alabama,167936,172789,10,12,17,17,7,14,6,7,...,1,6,33,23,1,7,25,1,0,21
Alaska,94809,135757,14,18,21,16,4,10,3,3,...,37,5,0,1,4,17,1,10,20,26
Arizona,591605,460685,16,16,16,15,6,12,6,5,...,1,27,3,11,1,18,13,4,18,33
Arkansas,150342,228830,13,16,16,16,8,11,6,5,...,0,48,25,8,5,3,0,8,0,20
California,590116,462341,12,14,16,14,6,13,6,7,...,5,3,9,5,5,13,9,18,29,44
Colorado,232219,188229,11,13,16,15,7,14,6,7,...,3,4,4,18,2,14,6,11,34,41
Connecticut,267941,294360,13,14,18,15,7,12,6,6,...,0,5,14,8,5,13,7,8,36,50
Delaware,191087,223006,12,13,16,14,7,11,8,8,...,6,15,0,25,5,22,0,21,3,27
District of Columbia,606015,435044,4,5,8,11,6,17,13,16,...,1,1,3,7,2,17,14,19,32,48


My goal is to now do this process for every dataset I have access to... which would be easy with a function.

In [108]:
def avg_commute(path):
    ### read in data
    data = pd.read_csv(path)
    
    
    ### get rid of specific columns
    print('Get rid of specific columns')
    pattern1 = r'Margin of Error'
    pattern2 = r'PERCENT ALLOCATED'
    pattern3 = r'Unnamed:'

    drop = []
    
    for col in data.columns:
        if (re.search(pattern1,col) or re.search(pattern2,col) or re.search(pattern3,col)):
            drop.append(col)
    #end
    
    data = data.drop(columns = drop)
    
    ### clean up columns names
    print('Clean up columns names')
    new_cols = []

    for col in data.columns:
        new_col = col.replace('Estimate!!Total!!', '').replace('!!',' ').strip()
        new_cols.append(new_col)
    #end
    
    data.columns = new_cols
    
    
    ### keep specific commute related columns
    print('Keep specific commute related columns')

    pattern = r'TRAVEL TIME'
    drop = []

    for col in data.columns:
        if (not re.search(pattern,col)
            and ((col != 'Geographic Area Name') 
            and (col != 'Workers 16 years and over')
            and (col != 'Workers 16 years and over who did not work from home')
            and (col != 'Estimate Car, truck, or van -- drove alone Workers 16 years and over')
            and (col != 'Estimate Car, truck, or van -- carpooled Workers 16 years and over')
            and (col != 'Estimate Public transportation (excluding taxicab) Workers 16 years and over'))):
            drop.append(col)
    #end

    data = data.drop(columns = drop)

    
    ### Set county as index
    print('Set county as index')

    data = data.set_index('Geographic Area Name')
    
    
    ### replace N and - with NaN
    print('Replace N and - with NaN')
    data = data.replace(to_replace='N',value=np.nan).replace(to_replace = '-', value = np.nan).replace('**',np.nan)
    
    
    ### remove rows that are mostly null or where 'Workers 16 years and over who did not work from home' is null
    ### ALSO CHECK IF THIS COLUMN EXISTS!
    print('Determine j')
    data = data.dropna(thresh = 30)
    
    if 'Workers 16 years and over who did not work from home' in data.columns:
        j = 1 # this indicates that my original code will work with this data
        
        drop = []
        for ind, values in data.iterrows():
            if pd.isna(values['Workers 16 years and over who did not work from home']):
                drop.append(ind)
                
    elif 'Workers 16 years and over who did not work at home' in data.columns:
        j = 2 # this indicates that my original code will *almost* work with this data
        
        drop = []
        for ind, values in data.iterrows():
            if pd.isna(values['Workers 16 years and over who did not work at home']):
                drop.append(ind)
    else:
        j = 0
        drop = []
    #end
    
    data = data.drop(index = drop)
    
    
    ### make dtypes numeric
    data = data.apply(pd.to_numeric)
    
    
    ### get rid of percentages by translating them to integers BASED ON j
    if j == 0:
        for row in data.itertuples():
            comm_total  = row[1]
            drove_alone = row[2]
            carpool     = row[3]
            pub_transit = row[4]

            for i in range(5,41):
                if i%4 == 1: # total
                    try:
                        new_val = int(comm_total*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                if i%4 == 2: # drove alone
                    try:
                        new_val = int(drove_alone*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                if i%4 == 3: # carpool
                    try:
                        new_val = int(carpool*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                if i%4 == 0: # public transit
                    try:
                        new_val = int(pub_transit*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
            #end
    elif j == 1:
        for row in data.itertuples():
            comm_total  = row[2]
            drove_alone = row[13]
            carpool     = row[24]
            pub_transit = row[35]

            for i in range(3,12):
                try:
                    new_val = int(comm_total*(row[i]/100))
                    data.loc[row.Index, data.columns[i-1]] = new_val
                except:
                    data.loc[row.Index, data.columns[i-1]] = np.nan


            for i in range(14,23):
                try:
                    new_val = int(drove_alone*(row[i]/100))
                    data.loc[row.Index, data.columns[i-1]] = new_val
                except:
                    data.loc[row.Index, data.columns[i-1]] = np.nan


            for i in range(25,34):
                try:
                    new_val = int(carpool*(row[i]/100))
                    data.loc[row.Index, data.columns[i-1]] = new_val
                except:
                    data.loc[row.Index, data.columns[i-1]] = np.nan


            for i in range(36,45):
                try:
                    new_val = int(pub_transit*(row[i]/100))
                    data.loc[row.Index, data.columns[i-1]] = new_val
                except:
                    data.loc[row.Index, data.columns[i-1]] = np.nan
        #end
    
    
    ### Clean up column names some more
    pattern1 = r'Workers 16 years and over who did not work from home'
    pattern2 = r'Estimate'
    pattern3 = r'(excluding taxicab)'

    new_cols = {}

    for col in data.columns:
        if (col != 'Workers 16 years and over who did not work from home') and (re.search(pattern1,col) 
                                                                                or re.search(pattern2,col) 
                                                                                or re.search(pattern3,col)):
            new_col = col.replace(str(pattern1),'').replace(str(pattern2),'').replace(str(pattern3),'').strip()
            new_cols[col] = new_col
    #end
    
    data = data.rename(columns = new_cols)
    
    
    ### set up aggregations for group by
    aggs = {}
    
    for col in data.columns:        
        num = data.columns.get_loc(col)
        
        if j == 0:
            if num < 4:
                aggs[col] = np.sum
            else:
                aggs[col] = np.mean
            #end
        elif j == 1:
            if (num < 2 or num == 12 or num == 23 or num == 34):
                aggs[col] = np.sum
            else:
                aggs[col] = np.mean
    #end
    
    
    ### add State column (so we can group by it)
    states = []

    for ind, values in data.iterrows():
        state = ind.split(', ')[1]
        states.append(state)
    #end
    
    data['State'] = states
    
    
    ### finally, group by state and return df
    
    data = data.groupby('State').agg(aggs).replace(np.nan, 0).astype(int)#.replace(to_replace=0,value=np.nan)
    
    return data

In [48]:
df = avg_commute('../data/census_data/2021_Data.csv')
df

,Workers 16 years and over,Workers 16 years and over who did not work from home,TRAVEL TIME TO WORK Less than 10 minutes,TRAVEL TIME TO WORK 10 to 14 minutes,TRAVEL TIME TO WORK 15 to 19 minutes,TRAVEL TIME TO WORK 20 to 24 minutes,TRAVEL TIME TO WORK 25 to 29 minutes,TRAVEL TIME TO WORK 30 to 34 minutes,TRAVEL TIME TO WORK 35 to 44 minutes,TRAVEL TIME TO WORK 45 to 59 minutes,...,Public transportation TRAVEL TIME TO WORK Less than 10 minutes,Public transportation TRAVEL TIME TO WORK 10 to 14 minutes,Public transportation TRAVEL TIME TO WORK 15 to 19 minutes,Public transportation TRAVEL TIME TO WORK 20 to 24 minutes,Public transportation TRAVEL TIME TO WORK 25 to 29 minutes,Public transportation TRAVEL TIME TO WORK 30 to 34 minutes,Public transportation TRAVEL TIME TO WORK 35 to 44 minutes,Public transportation TRAVEL TIME TO WORK 45 to 59 minutes,Public transportation TRAVEL TIME TO WORK 60 or more minutes,Public transportation TRAVEL TIME TO WORK Mean travel time to work (minutes)
State,,,,,,,,,,,,,,,,,,,,,
Alabama,965107,863949,15013,19791,28402,31463,13064,27249,12756,13684,...,28,125,52,155,21,42,38,0,0,22
Alaska,154092,135757,21042,26608,29187,21178,7330,10724,5294,5294,...,395,224,0,50,86,284,62,0,779,35
Arizona,2958027,2303425,53808,58428,69679,76983,37033,69401,34021,34735,...,181,157,157,530,223,1089,711,605,2033,33
Arkansas,254012,228830,24255,33180,37299,37528,17619,33180,16933,17162,...,0,71,547,0,0,83,0,225,0,24
California,17113392,13407902,44631,57945,72445,70569,30604,70927,31992,36341,...,254,402,690,866,380,2208,1217,2055,4544,44
Colorado,2510638,1882291,20715,24055,30693,30149,14245,29824,12943,14110,...,71,48,192,324,61,519,299,488,1047,41
Connecticut,1469099,1177440,34536,41540,52262,45099,23563,38763,20381,19050,...,70,558,606,650,286,1005,416,702,2572,58
Delaware,288135,223006,23192,29882,41256,33450,18955,27429,16948,18732,...,675,254,27,863,581,420,0,2325,387,33
District of Columbia,606015,435044,19141,22187,36543,50465,26537,77437,57860,71347,...,886,1370,2740,5883,2176,14104,11767,15313,26354,48


The first few times I ran this, the number of people driving alone would be larger than the total of people who didn't work from home... which doesn't make any sense. I had to investigate for a bit, but I found out that there were several rows that had NaN for total workers not working at home but some number for driving alone. I decided to remove these columns to make the data as uniform as possible and have as few errors as possible (even though some data is now useless, but I believe it's worth it).

I first checked to see if using a list instead of an index would work.

In [33]:
list(commutes_avg.columns[0:2])

['Workers 16 years and over',
 'Workers 16 years and over who did not work from home']

But it did not.

I then checked to see where total workers (including at home workers) was null.

In [34]:
commutes.loc[pd.isna(commutes['Workers 16 years and over'])]

,Workers 16 years and over,Workers 16 years and over who did not work from home,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,...,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 60 or more minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes),State
Geographic Area Name,,,,,,,,,,,,,,,,,,,,,
"Elmore County, Alabama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama
"Limestone County, Alabama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama
"Jefferson County, Arkansas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arkansas
"Tehama County, California",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,California
"Nassau County, Florida",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Florida
"Putnam County, Florida",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Florida
"Walton County, Florida",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Florida
"Catoosa County, Georgia",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Georgia
"Effingham County, Georgia",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Georgia


But silly me, that's not the column I was interested in nor the table I needed to check! I think this is why I was going in circles for a bit.

I decided to see if there was any row where this contradiciton happened (more drivers than workers travelling).

In [35]:
commutes_avg.loc[commutes_avg['Car, truck, or van -- drove alone Workers 16 years and over']
            > commutes_avg['Workers 16 years and over who did not work from home']]

,Workers 16 years and over,Workers 16 years and over who did not work from home,TRAVEL TIME TO WORK Less than 10 minutes,TRAVEL TIME TO WORK 10 to 14 minutes,TRAVEL TIME TO WORK 15 to 19 minutes,TRAVEL TIME TO WORK 20 to 24 minutes,TRAVEL TIME TO WORK 25 to 29 minutes,TRAVEL TIME TO WORK 30 to 34 minutes,TRAVEL TIME TO WORK 35 to 44 minutes,TRAVEL TIME TO WORK 45 to 59 minutes,...,Public transportation TRAVEL TIME TO WORK Less than 10 minutes,Public transportation TRAVEL TIME TO WORK 10 to 14 minutes,Public transportation TRAVEL TIME TO WORK 15 to 19 minutes,Public transportation TRAVEL TIME TO WORK 20 to 24 minutes,Public transportation TRAVEL TIME TO WORK 25 to 29 minutes,Public transportation TRAVEL TIME TO WORK 30 to 34 minutes,Public transportation TRAVEL TIME TO WORK 35 to 44 minutes,Public transportation TRAVEL TIME TO WORK 45 to 59 minutes,Public transportation TRAVEL TIME TO WORK 60 or more minutes,Public transportation TRAVEL TIME TO WORK Mean travel time to work (minutes)
State,,,,,,,,,,,,,,,,,,,,,


Nothing... that's wasn't very helpful.

Since I had come up with the separate aggregations for grouping by within the function, I tried it out here to see if anything changed.

In [36]:
aggs = {}

for col in commutes_nan.columns:
    num = commutes_nan.columns.get_loc(col)
    if (num < 2 or num == 12 or num == 23 or num == 34):
        aggs[col] = np.sum
    elif num < 45:
        aggs[col] = np.mean
#end

In [37]:
commutes_agg = commutes_nan.groupby('State').agg(aggs)
#commutes_agg

But nothing changed.

I then checked the df I brought in with the function to see if *that* did anything.

In [38]:
df.loc[df['Estimate Car, truck, or van -- drove alone Workers 16 years and over']
            > df['Workers 16 years and over who did not work from home']]

,Workers 16 years and over,Workers 16 years and over who did not work from home,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,...,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 60 or more minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes)
State,,,,,,,,,,,,,,,,,,,,,


And of course it didn't.

Finally, I decided to replace all NaN's with 0's and then compare. Maybe this would get me somewhere...

In [39]:
commutes_zero = commutes_nan.replace(to_replace=np.nan, value = 0)
#commutes_zero

In [40]:
commutes_zero.loc[commutes_zero['Estimate Car, truck, or van -- drove alone Workers 16 years and over']
            > commutes_zero['Workers 16 years and over who did not work from home']]

,Workers 16 years and over,Workers 16 years and over who did not work from home,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,...,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 10 to 14 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 15 to 19 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 20 to 24 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 25 to 29 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 30 to 34 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 35 to 44 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 45 to 59 minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK 60 or more minutes,Estimate Public transportation (excluding taxicab) Workers 16 years and over who did not work from home TRAVEL TIME TO WORK Mean travel time to work (minutes),State
Geographic Area Name,,,,,,,,,,,,,,,,,,,,,
"Shelby County, Alabama",98228.0,0.0,14.2,12.4,14.1,14.7,7.2,16.2,6.6,10.2,...,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alabama
"Tuscaloosa County, Alabama",112218.0,0.0,11.1,12.9,23.5,15.3,5.9,8.9,4.6,8.0,...,0.0,51.6,40.0,0.0,0.0,0.0,8.4,0.0,19.5,Alabama
"Matanuska-Susitna Borough, Alaska",35527.0,0.0,13.1,17.6,20.7,18.1,4.4,12.6,3.2,3.8,...,0.0,0.0,0.0,4.8,20.5,0.0,20.5,0.0,18.8,Alaska
"Benton County, Arkansas",145521.0,0.0,13.6,14.1,16.7,18.7,9.1,13.0,6.3,4.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Arkansas
"Sebastian County, Arkansas",71596.0,0.0,14.4,18.2,17.7,13.5,9.2,9.2,6.8,5.3,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Arkansas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Skagit County, Washington",52095.0,0.0,18.8,15.8,11.4,10.3,5.3,13.6,8.7,9.6,...,0.0,0.0,51.7,0.0,0.0,0.0,48.3,0.0,0.0,Washington
"Monongalia County, West Virginia",63999.0,0.0,12.1,14.3,18.3,13.8,7.6,11.2,7.5,6.6,...,0.0,17.7,0.0,33.5,40.9,0.0,0.0,7.9,28.0,West Virginia
"Washington County, Wisconsin",63853.0,0.0,17.1,15.8,17.4,17.1,6.6,10.9,6.3,4.5,...,0.0,0.0,91.4,0.0,0.0,0.0,8.6,0.0,0.0,Wisconsin


#### *AHA!* There's the problem!

Many of these rows didn't have anything listed for workers who had to travel, but whenever I would try comparing them numerically, nothing showed up because they were NaN: 'Not a Number'

So with this in mind, I went to remove any row where this column was null. That was able to fix it :)

The last step for this notebook is to clean each table and save it so we can pull it into a fresh new notebook.

In [106]:
for i in range (2010,2023):
    if i != 2020:
        print(i)
        
        try:
            path = f'../data/census_data/{i}_Data.csv'
            new_df = avg_commute(path)
            new_df.to_csv(f'../data/cleaned_data/{i}_data_cleaned.csv')
            print('Completed')
        except:
            print('Skipped')
            continue
#end

2010
j is 0
Completed
2011
j is 0
Completed
2012
j is 0
Completed
2013
j is 0
Completed
2014
j is 0
Completed
2015
j is 0
Completed
2016
j is 0
Completed
2017
j is 0
Completed
2018
Skipped
2019
j is 1
Completed
2021
j is 1
Completed
2022
j is 1
Completed


What when wrong? It's time to take a closer look at each one.

In [60]:
df = avg_commute('../data/census_data/2010_Data.csv')

KeyError: 'Workers 16 years and over who did not work from home'

In [96]:
data = pd.read_csv('../data/census_data/2010_Data.csv')
#data

In [75]:
pd.set_option('display.max_columns', None)

In [97]:
### get rid of specific columns
pattern1 = r'Margin of Error'
pattern2 = r'PERCENT ALLOCATED'
pattern3 = r'Unnamed:'

drop = []
  
for col in data.columns:
    if (re.search(pattern1,col) or re.search(pattern2,col) or re.search(pattern3,col)):
        drop.append(col)
#end
    
data = data.drop(columns = drop)
 
### clean up columns names
new_cols = []

for col in data.columns:
    new_col = col.replace('Estimate', '').replace('Total','').replace('!!',' ').replace('  ',' ').strip()
    new_cols.append(new_col)
#end
    
data.columns = new_cols


### keep specific commute related columns
pattern = r'TRAVEL TIME'
drop = []

for col in data.columns:
    if (not re.search(pattern,col)
        and ((col != 'Geographic Area Name') 
        and (col != 'Workers 16 years and over')
        and (col != 'Workers 16 years and over who did not work from home')
        and (col != 'Car, truck, or van -- drove alone Workers 16 years and over')
        and (col != 'Car, truck, or van -- carpooled Workers 16 years and over')
        and (col != 'Public transportation (excluding taxicab) Workers 16 years and over'))):
        drop.append(col)
#end

data = data.drop(columns = drop)


### Set county as index
data = data.set_index('Geographic Area Name')

    
### replace N and - with NaN
data = data.replace(to_replace='N',value=np.nan).replace(to_replace = '-', value = np.nan).replace('**',np.nan)
    
    
### remove rows that are mostly null or where 'Workers 16 years and over who did not work from home' is null
data = data.dropna(thresh = 30)

drop = []
if 'Workers 16 years and over who did not work from home' in data.columns:
    j = 1 # this indicates that my original code will work with this data
        
    for ind, values in data.iterrows():
        if pd.isna(values['Workers 16 years and over who did not work from home']):
            drop.append(ind)
            
elif 'Workers 16 years and over who did not work at home' in data.columns:
    j = 2 # this indicates that my original code will *almost* work with this data
        
    for ind, values in data.iterrows():
        if pd.isna(values['Workers 16 years and over who did not work at home']):
            drop.append(ind)
else:
    j = 0
#end
    
data = data.drop(index = drop)
    
    
### make dtypes numeric
data = data.apply(pd.to_numeric)

print('j is',j)

data

j is 0


,Workers 16 years and over,"Car, truck, or van -- drove alone Workers 16 years and over","Car, truck, or van -- carpooled Workers 16 years and over",Public transportation (excluding taxicab) Workers 16 years and over,TRAVEL TIME TO WORK Less than 10 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK Less than 10 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK Less than 10 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK Less than 10 minutes,TRAVEL TIME TO WORK 10 to 14 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 10 to 14 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 10 to 14 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 10 to 14 minutes,TRAVEL TIME TO WORK 15 to 19 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 15 to 19 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 15 to 19 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 15 to 19 minutes,TRAVEL TIME TO WORK 20 to 24 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 20 to 24 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 20 to 24 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 20 to 24 minutes,TRAVEL TIME TO WORK 25 to 29 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 25 to 29 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 25 to 29 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 25 to 29 minutes,TRAVEL TIME TO WORK 30 to 34 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 30 to 34 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 30 to 34 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 30 to 34 minutes,TRAVEL TIME TO WORK 35 to 44 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 35 to 44 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 35 to 44 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 35 to 44 minutes,TRAVEL TIME TO WORK 45 to 59 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 45 to 59 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 45 to 59 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 45 to 59 minutes,TRAVEL TIME TO WORK 60 or more minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 60 or more minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 60 or more minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 60 or more minutes,TRAVEL TIME TO WORK Mean travel time to work (minutes),"Car, truck, or van -- drove alone TRAVEL TIME TO WORK Mean travel time to work (minutes)","Car, truck, or van -- carpooled TRAVEL TIME TO WORK Mean travel time to work (minutes)",Public transportation (excluding taxicab) TRAVEL TIME TO WORK Mean travel time to work (minutes)
Geographic Area Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Calhoun County, Alabama",43874,38242,4003,136,14.7,14.5,8.5,0.0,17.8,18.7,14.2,0.0,17.3,15.8,35.6,0.0,13.5,14.1,10.0,0.0,7.7,7.9,7.2,0.0,12.4,12.2,7.6,100.0,5.3,5.8,2.4,0.0,6.9,7.2,4.9,0.0,4.4,3.8,9.6,0.0,21.4,21.3,22.7,NaN
"Houston County, Alabama",50739,45459,3365,113,14.7,15.0,8.9,0.0,21.6,21.5,20.1,0.0,18.7,18.3,26.3,19.5,14.9,15.5,8.3,0.0,4.1,4.3,2.7,0.0,14.2,14.5,13.0,57.5,3.8,3.7,7.0,0.0,5.3,5.0,7.6,0.0,2.6,2.3,6.0,23.0,20.3,19.9,25.6,47.8
"Jefferson County, Alabama",350866,298194,36354,2573,7.2,6.7,7.8,0.0,11.6,11.7,9.5,5.0,15.8,16.2,12.3,16.1,18.2,18.5,19.5,2.7,6.4,6.7,3.4,5.5,17.2,17.4,17.4,16.8,7.3,7.3,8.7,5.0,9.0,9.0,9.7,16.4,7.3,6.5,11.7,32.6,27.0,26.7,29.7,44.5
"Lee County, Alabama",53658,40828,7187,478,19.6,18.2,16.1,0.0,19.5,20.2,13.6,21.5,18.8,17.5,21.4,68.8,13.0,13.8,13.6,0.0,4.9,6.0,1.3,0.0,9.9,11.4,4.8,6.1,4.3,4.0,7.4,3.6,5.3,5.2,9.4,0.0,4.7,3.6,12.3,0.0,20.9,20.0,29.9,15.5
"Madison County, Alabama",196561,167379,17905,776,9.5,9.6,8.9,5.9,11.7,11.5,13.8,0.0,18.8,18.6,12.6,33.4,19.5,20.4

In [67]:
list(data.columns)

['Geography',
 'Geographic Area Name',
 'Total Estimate Workers 16 years and over',
 'Car, truck, or van -- drove alone Estimate Workers 16 years and over',
 'Car, truck, or van -- carpooled Estimate Workers 16 years and over',
 'Public transportation (excluding taxicab) Estimate Workers 16 years and over',
 'Total Estimate AGE 16 to 19 years',
 'Car, truck, or van -- drove alone Estimate AGE 16 to 19 years',
 'Car, truck, or van -- carpooled Estimate AGE 16 to 19 years',
 'Public transportation (excluding taxicab) Estimate AGE 16 to 19 years',
 'Total Estimate AGE 20 to 24 years',
 'Car, truck, or van -- drove alone Estimate AGE 20 to 24 years',
 'Car, truck, or van -- carpooled Estimate AGE 20 to 24 years',
 'Public transportation (excluding taxicab) Estimate AGE 20 to 24 years',
 'Total Estimate AGE 25 to 44 years',
 'Car, truck, or van -- drove alone Estimate AGE 25 to 44 years',
 'Car, truck, or van -- carpooled Estimate AGE 25 to 44 years',
 'Public transportation (excluding taxi

**Interesting Note:** this dataset does not have workers *who did not work from home* listed. I guess that makes sense because remote work was not nearly as common then as it is now (particularly because of the pandemic). Because of this difference, I'll have to adjust the function to account for this missing column.

In [93]:
data

,Workers 16 years and over,"Car, truck, or van -- drove alone Workers 16 years and over","Car, truck, or van -- carpooled Workers 16 years and over",Public transportation (excluding taxicab) Workers 16 years and over,TRAVEL TIME TO WORK Less than 10 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK Less than 10 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK Less than 10 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK Less than 10 minutes,TRAVEL TIME TO WORK 10 to 14 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 10 to 14 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 10 to 14 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 10 to 14 minutes,TRAVEL TIME TO WORK 15 to 19 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 15 to 19 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 15 to 19 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 15 to 19 minutes,TRAVEL TIME TO WORK 20 to 24 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 20 to 24 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 20 to 24 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 20 to 24 minutes,TRAVEL TIME TO WORK 25 to 29 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 25 to 29 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 25 to 29 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 25 to 29 minutes,TRAVEL TIME TO WORK 30 to 34 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 30 to 34 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 30 to 34 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 30 to 34 minutes,TRAVEL TIME TO WORK 35 to 44 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 35 to 44 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 35 to 44 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 35 to 44 minutes,TRAVEL TIME TO WORK 45 to 59 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 45 to 59 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 45 to 59 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 45 to 59 minutes,TRAVEL TIME TO WORK 60 or more minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 60 or more minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 60 or more minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 60 or more minutes,TRAVEL TIME TO WORK Mean travel time to work (minutes),"Car, truck, or van -- drove alone TRAVEL TIME TO WORK Mean travel time to work (minutes)","Car, truck, or van -- carpooled TRAVEL TIME TO WORK Mean travel time to work (minutes)",Public transportation (excluding taxicab) TRAVEL TIME TO WORK Mean travel time to work (minutes)
Geographic Area Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Calhoun County, Alabama",43874,38242,4003,136,14.7,14.5,8.5,0.0,17.8,18.7,14.2,0.0,17.3,15.8,35.6,0.0,13.5,14.1,10.0,0.0,7.7,7.9,7.2,0.0,12.4,12.2,7.6,100.0,5.3,5.8,2.4,0.0,6.9,7.2,4.9,0.0,4.4,3.8,9.6,0.0,21.4,21.3,22.7,NaN
"Houston County, Alabama",50739,45459,3365,113,14.7,15.0,8.9,0.0,21.6,21.5,20.1,0.0,18.7,18.3,26.3,19.5,14.9,15.5,8.3,0.0,4.1,4.3,2.7,0.0,14.2,14.5,13.0,57.5,3.8,3.7,7.0,0.0,5.3,5.0,7.6,0.0,2.6,2.3,6.0,23.0,20.3,19.9,25.6,47.8
"Jefferson County, Alabama",350866,298194,36354,2573,7.2,6.7,7.8,0.0,11.6,11.7,9.5,5.0,15.8,16.2,12.3,16.1,18.2,18.5,19.5,2.7,6.4,6.7,3.4,5.5,17.2,17.4,17.4,16.8,7.3,7.3,8.7,5.0,9.0,9.0,9.7,16.4,7.3,6.5,11.7,32.6,27.0,26.7,29.7,44.5
"Lee County, Alabama",53658,40828,7187,478,19.6,18.2,16.1,0.0,19.5,20.2,13.6,21.5,18.8,17.5,21.4,68.8,13.0,13.8,13.6,0.0,4.9,6.0,1.3,0.0,9.9,11.4,4.8,6.1,4.3,4.0,7.4,3.6,5.3,5.2,9.4,0.0,4.7,3.6,12.3,0.0,20.9,20.0,29.9,15.5
"Madison County, Alabama",196561,167379,17905,776,9.5,9.6,8.9,5.9,11.7,11.5,13.8,0.0,18.8,18.6,12.6,33.4,19.5,20.4

In [98]:
for row in data.itertuples():
    comm_total  = row[1]
    drove_alone = row[2]
    carpool     = row[3]
    pub_transit = row[4]
    
    for i in range(5,41):
        if i%4 == 1: # total
            try:
                new_val = int(comm_total*(row[i]/100))
                data.loc[row.Index, data.columns[i-1]] = new_val
            except:
                data.loc[row.Index, data.columns[i-1]] = np.nan
            #end
        if i%4 == 2: # drove alone
            try:
                new_val = int(drove_alone*(row[i]/100))
                data.loc[row.Index, data.columns[i-1]] = new_val
            except:
                data.loc[row.Index, data.columns[i-1]] = np.nan
            #end
        if i%4 == 3: # carpool
            try:
                new_val = int(carpool*(row[i]/100))
                data.loc[row.Index, data.columns[i-1]] = new_val
            except:
                data.loc[row.Index, data.columns[i-1]] = np.nan
            #end
        if i%4 == 0: # public transit
            try:
                new_val = int(pub_transit*(row[i]/100))
                data.loc[row.Index, data.columns[i-1]] = new_val
            except:
                data.loc[row.Index, data.columns[i-1]] = np.nan
            #end

This code helped for the most part, but 2015 and 2018 are being difficult.

In [99]:
data

,Workers 16 years and over,"Car, truck, or van -- drove alone Workers 16 years and over","Car, truck, or van -- carpooled Workers 16 years and over",Public transportation (excluding taxicab) Workers 16 years and over,TRAVEL TIME TO WORK Less than 10 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK Less than 10 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK Less than 10 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK Less than 10 minutes,TRAVEL TIME TO WORK 10 to 14 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 10 to 14 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 10 to 14 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 10 to 14 minutes,TRAVEL TIME TO WORK 15 to 19 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 15 to 19 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 15 to 19 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 15 to 19 minutes,TRAVEL TIME TO WORK 20 to 24 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 20 to 24 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 20 to 24 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 20 to 24 minutes,TRAVEL TIME TO WORK 25 to 29 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 25 to 29 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 25 to 29 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 25 to 29 minutes,TRAVEL TIME TO WORK 30 to 34 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 30 to 34 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 30 to 34 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 30 to 34 minutes,TRAVEL TIME TO WORK 35 to 44 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 35 to 44 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 35 to 44 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 35 to 44 minutes,TRAVEL TIME TO WORK 45 to 59 minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 45 to 59 minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 45 to 59 minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 45 to 59 minutes,TRAVEL TIME TO WORK 60 or more minutes,"Car, truck, or van -- drove alone TRAVEL TIME TO WORK 60 or more minutes","Car, truck, or van -- carpooled TRAVEL TIME TO WORK 60 or more minutes",Public transportation (excluding taxicab) TRAVEL TIME TO WORK 60 or more minutes,TRAVEL TIME TO WORK Mean travel time to work (minutes),"Car, truck, or van -- drove alone TRAVEL TIME TO WORK Mean travel time to work (minutes)","Car, truck, or van -- carpooled TRAVEL TIME TO WORK Mean travel time to work (minutes)",Public transportation (excluding taxicab) TRAVEL TIME TO WORK Mean travel time to work (minutes)
Geographic Area Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Calhoun County, Alabama",43874,38242,4003,136,6449.0,5545.0,340.0,0.0,7809.0,7151.0,568.0,0.0,7590.0,6042.0,1425.0,0.0,5922.0,5392.0,400.0,0.0,3378.0,3021.0,288.0,0.0,5440.0,4665.0,304.0,136.0,2325.0,2218.0,96.0,0.0,3027.0,2753.0,196.0,0.0,1930.0,1453.0,384.0,0.0,21.4,21.3,22.7,NaN
"Houston County, Alabama",50739,45459,3365,113,7458.0,6818.0,299.0,0.0,10959.0,9773.0,676.0,0.0,9488.0,8318.0,884.0,22.0,7560.0,7046.0,279.0,0.0,2080.0,1954.0,90.0,0.0,7204.0,6591.0,437.0,64.0,1928.0,1681.0,235.0,0.0,2689.0,2272.0,255.0,0.0,1319.0,1045.0,201.0,25.0,20.3,19.9,25.6,47.8
"Jefferson County, Alabama",350866,298194,36354,2573,25262.0,19978.0,2835.0,0.0,40700.0,34888.0,3453.0,128.0,55436.0,48307.0,4471.0,414.0,63857.0,55165.0,7089.0,69.0,22455.0,19978.0,1236.0,141.0,60348.0,51885.0,6325.0,432.0,25613.0,21768.0,3162.0,128.0,31577.0,26837.0,3526.0,421.0,25613.0,19382.0,4253.0,838.0,27.0,26.7,29.7,44.5
"Lee County, Alabama",53658,40828,7187,478,10516.0,7430.0,1157.0,0.0,10463.0,8247.0,977.0,102.0,10087.0,7144.0,1538.0,328

In [118]:
data = pd.read_csv('../data/census_data/2018_Data.csv', header = 0)

In [114]:
data_csv = avg_commute('../data/census_data/2018_Data.csv')
data_csv

Get rid of specific columns
Clean up columns names
Keep specific commute related columns
Set county as index
Replace N and - with NaN
Determine j
j is 0


,Workers 16 years and over,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK Less than 10 minutes,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 10 to 14 minutes,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 15 to 19 minutes,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 20 to 24 minutes,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 25 to 29 minutes,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 30 to 34 minutes,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 35 to 44 minutes,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 45 to 59 minutes,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 60 or more minutes,Workers 16 years and over who did not work at home TRAVEL TIME TO WORK Mean travel time to work (minutes),"Car, truck, or van -- drove alone Workers 16 years and over","Car, truck, or van -- carpooled Workers 16 years and over",Public transportation Workers 16 years and over,"Car, truck, or van -- drove alone Workers 16 years and over who did not work at home TRAVEL TIME TO WORK Less than 10 minutes","Car, truck, or van -- carpooled Workers 16 years and over who did not work at home TRAVEL TIME TO WORK Less than 10 minutes",Public transportation Workers 16 years and over who did not work at home TRAVEL TIME TO WORK Less than 10 minutes,"Car, truck, or van -- drove alone Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 10 to 14 minutes","Car, truck, or van -- carpooled Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 10 to 14 minutes",Public transportation Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 10 to 14 minutes,"Car, truck, or van -- drove alone Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 15 to 19 minutes","Car, truck, or van -- carpooled Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 15 to 19 minutes",Public transportation Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 15 to 19 minutes,"Car, truck, or van -- drove alone Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 20 to 24 minutes","Car, truck, or van -- carpooled Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 20 to 24 minutes",Public transportation Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 20 to 24 minutes,"Car, truck, or van -- drove alone Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 25 to 29 minutes","Car, truck, or van -- carpooled Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 25 to 29 minutes",Public transportation Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 25 to 29 minutes,"Car, truck, or van -- drove alone Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 30 to 34 minutes","Car, truck, or van -- carpooled Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 30 to 34 minutes",Public transportation Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 30 to 34 minutes,"Car, truck, or van -- drove alone Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 35 to 44 minutes","Car, truck, or van -- carpooled Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 35 to 44 minutes",Public transportation Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 35 to 44 minutes,"Car, truck, or van -- drove alone Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 45 to 59 minutes","Car, truck, or van -- carpooled Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 45 to 59 minutes",Public transportation Workers 16 years and over who did not work at home TRAVEL TIME TO WORK 45 to 59 minutes,"Car, truck, or van -- drove alone Work

In [119]:
data

Geography                  Geographic Area Name  \
0    0500000US01003               Baldwin County, Alabama   
1    0500000US01015               Calhoun County, Alabama   
2    0500000US01043               Cullman County, Alabama   
3    0500000US01049                DeKalb County, Alabama   
4    0500000US01051                Elmore County, Alabama   
..              ...                                   ...   
833  0500000US72113          Ponce Municipio, Puerto Rico   
834  0500000US72127       San Juan Municipio, Puerto Rico   
835  0500000US72135       Toa Alta Municipio, Puerto Rico   
836  0500000US72137       Toa Baja Municipio, Puerto Rico   
837  0500000US72139  Trujillo Alto Municipio, Puerto Rico   

    Estimate!!Total!!Workers 16 years and over  \
0                                        83189   
1                                        47088   
2                                        30802   
3                                            N   
4                                            N   
..                                         ...   
833                                      53865   
834                                     227452   
835                                          N   
836                                          N   
837                                      11116   

    Margin of Error!!Total MOE!!Workers 16 years and over  \
0                                                 4238      
1                                                 3321      
2                                                 2202      
3                                                    N      
4                                                    N      
..                                                 ...      
833                                               3677      
834                                               7963      
835                                                  N      
836                                                  N      
837                                               2339      

    Estimate!!Total!!Workers 16 years and over!!AGE!!16 to 19 years  \
0                                                  5.6                
1                                                    N                
2                                                    N                
3                                                    N                
4                                                    N                
..                                                 ...                
833                                                1.0                
834                                                1.4                
835                                                  N                
836                                                  N                
837                                                  N                

    Margin of Error!!Total MOE!!Workers 16 years and over!!AGE!!16 to 19 years  \
0                                                  1.2                           
1                                                    N                           
2                                                    N                           
3                                                    N                           
4                                                    N                           
..                                                 ...                           
833                                                0.7                           
834                                                0.4                           
835                                                  N                           
836                                                  N                           
837                                                  N                           

    Estimate!!Total!!Workers 16 years and over!!AGE!!20 to 24 years  \
0                                 